<a href="https://colab.research.google.com/github/ahmadSum1/MIR_Machine_Learning_PARIS_Sebastien/blob/dev_ex02/MIRs1_ML_Exercice2_KCV_and_KNN_SVM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [63]:
import pandas as pd
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics
from sklearn.metrics import mean_absolute_error

import numpy as np
import matplotlib.pyplot as plt

if 'google.colab' in str(get_ipython()):
  from google.colab import files
  import io
else:
  print('Not running on CoLab')

data_file = 'ionosphere_data.csv'
df = pd.read_csv('ionosphere_data.csv')
# uploaded = files.upload() #Si google colab
# df = pd.read_csv(io.BytesIO(uploaded[data_file]))

In [64]:
# df

Since the columns b has no variance, it must be removed by:

In [65]:
df.drop(columns=['column_b'], inplace=True)
# df

The column_a with boolean must be encoded into 0/1 values by::

In [66]:
df['column_a'] = df.column_a.astype('float64')
# df

Labels must be extracted from the df object via:

In [67]:
df.rename(columns={'column_ai': 'label'}, inplace=True)
df['label'] = df.label.astype('category')
encoding = {'g': 1, 'b': 0}
df.label.replace(encoding, inplace=True)

Finally data are obtained via:

In [68]:
X = df.values[:, :-1]
y = df.values[:, -1]

In [69]:
print(X.shape)
print(y.shape)

(351, 33)
(351,)


In [70]:
def my_train_test_idx(X,y,K,train_size):
  N = X.shape[0]
  Ntrain = int(N* train_size)
  Ntest = N-Ntrain
  Itrain = np.zeros((Ntrain,K), dtype=int)
  Itest =  np.zeros((Ntest, K), dtype=int)
  for k in range(K):
    idxs = np.random.permutation(N)
    split_h_idx = Ntrain                          # integer for line selection (horizontal selection)
    train_idxs = idxs[ :split_h_idx]              # indexing/selection of the test %
    test_idxs = idxs[split_h_idx : ]              # indexing/selection of the remaining % for training
    # print(test_idxs)
    Itrain[:,k] = train_idxs
    Itest[:,k] = test_idxs
  return Itrain, Itest

In [71]:
Itrain, Itest = my_train_test_idx(X,y,10,0.6)

# Xtrain = X[Itrain[: , 1],:]
# ytrain = y[Itrain[: , 1]]
# Xtest = X[Itest[: , 1],:]
# ytest = y[Itest[: , 1]]

In [72]:
# Xtest

In [73]:
model = KNeighborsClassifier(n_neighbors=5)

In [76]:
K=10
error = 0
for i in range(K):
  #Create KNN Classifier
  knn = KNeighborsClassifier(n_neighbors=5)
  Xtrain = X[Itrain[: , i],:]
  ytrain = y[Itrain[: , i]] 
  Xtest = X[Itest[: , i],:]
  ytest = y[Itest[: , i]]
  #Train the model using the training sets
  knn.fit(Xtrain, ytrain)
  #Predict the response for test dataset
  ypred = knn.predict(Xtest)
  print("Accuracy of K=",i,": ",metrics.accuracy_score(ytest, ypred))
  error += mean_absolute_error(ytest, ypred)
mean_error = error/10
print ("Mean error over K fold:",mean_error)

Accuracy of K= 0 :  0.8723404255319149
Accuracy of K= 1 :  0.8652482269503546
Accuracy of K= 2 :  0.851063829787234
Accuracy of K= 3 :  0.851063829787234
Accuracy of K= 4 :  0.8226950354609929
Accuracy of K= 5 :  0.851063829787234
Accuracy of K= 6 :  0.8156028368794326
Accuracy of K= 7 :  0.8439716312056738
Accuracy of K= 8 :  0.8368794326241135
Accuracy of K= 9 :  0.8226950354609929
Mean error over K fold: 0.15673758865248227


In [75]:
def my_train_test_valid_idx(X,y,K,train_size,valid_size):
  N = X.shape[0]
  # Ntrain + Nvalid + Ntest = N,
  # Itrain(Ntrain x K), Ivalid(Nvalid x K) and Itest(Ntest x K).
  Ntrain = int(N* train_size)
  Nvalid = int((N-Ntrain)* train_size)
  Ntest = N - Ntrain - Nvalid
  
  Itrain = np.zeros( (Ntrain,K), dtype=int)
  Ivalid = np.zeros( (Nvalid,K), dtype=int)
  Itest  = np.zeros( (Ntest, K), dtype=int)
  for k in range(K):
    idxs = np.random.permutation(N)
    split_h_idx1 = Ntrain
    split_h_idx2 = Ntrain+Nvalid                          # integer for line selection (horizontal selection)
    train_idxs = idxs[ :split_h_idx]                      # indexing/selection of the test %
    valid_idxs = idxs[split_h_idx : split_h_idx2 ]        # indexing/selection of the remaining % for training
    test_idxs = idxs[split_h_idx2 : ]                     # indexing/selection of the remaining % for training
    # print(test_idxs)
    Itrain[:,k] = train_idxs
    Ivalid[:,k] = valid_idxs
    Itest[:,k]  = test_idxs
  return Itrain, Ivalid, Itest
